# Notebook de preparação do dataset de Acidentes

### Imports

In [ ]:
import json

import pandas as pd
from resolve_path import ajuste_path

### Leitura do dataset de entrada pre-processado

In [ ]:
path = "data/"
path = ajuste_path(path)

pathUtil = path + "util/"

df_acidentes_padronizado = pd.read_csv(
    pathUtil + "acidentes/" + "acidentes_preprocessado.csv", encoding="utf-8", delimiter="@"
)

df_acidentes_padronizado.head(10)

### Ordenando por tempo(mais antigo é o primeiro) e criando coluna data com nível de detalhe em hora

In [ ]:
# preenchendo com 12 horas o que está nan para não perder a singularidade da hora
df_acidentes_padronizado["hora"] = df_acidentes_padronizado["hora"].fillna(12)

valores = df_acidentes_padronizado["hora"].unique()

subs = {}
for hora in valores:
    try:
        subs[hora] = hora[:2]
    except:
        # subs['Hora'][np.nan] = 'nao informado'
        pass

print(json.dumps(subs, indent=4))

df_acidentes_padronizado["hora"] = df_acidentes_padronizado["hora"].replace(
    subs)

# Criando coluna "Data" a partir das colunas 'day', 'month' e 'year'
df_acidentes_padronizado['data'] = pd.to_datetime(
    df_acidentes_padronizado[['ano', 'mes', 'dia', 'hora']].rename(
        columns={'ano': 'year', 'mes': 'month', 'dia': 'day', 'hora': 'hour'}
    ))

# Ordenando o dataframe pelas datas, do mais antigo para o mais recente
df_acidentes_padronizado = df_acidentes_padronizado.sort_values(by='data')

# Renomeando as colunas de volta
df_acidentes_padronizado.rename(
    columns={'day': 'dia', 'month': 'mes', 'year': 'ano'}, inplace=True)

# Reset do index
df_acidentes_padronizado = df_acidentes_padronizado.reset_index(drop=True)

### Criando o indice dos acidentes originais antes de retirar algumas categorias

In [ ]:
# Criando uma nova coluna "Indice acidente" com o valor do índice de cada linha
df_acidentes_padronizado['indice acidente'] = [
    i for i in df_acidentes_padronizado.index]

### Removendo acidentes de terceiro

In [ ]:
print("Linhas antes da remoção de terceiros: ",
      df_acidentes_padronizado.shape[0])
qtd_linhas_com_terceiros = df_acidentes_padronizado[
    df_acidentes_padronizado["empregado"] == "terceiro"
].shape[0]

print("Removendo " + str(qtd_linhas_com_terceiros) + " linhas com terceiros")

df_acidentes_padronizado = df_acidentes_padronizado[
    df_acidentes_padronizado['empregado'] == 'proprio']

print("Linhas após a remoção de terceiros: ",
      df_acidentes_padronizado.shape[0])

### Removendo linhas cuja Classificação é de trajeto

In [ ]:
# contando quantas linhas possuem "trajeto" na coluna Classificação
conta_linhas_trajeto = (
    df_acidentes_padronizado["classificacao"].str.contains("trajeto").sum()
)

print(
    f"Quantidade de linhas que possuem 'trajeto' na coluna Classificação: {conta_linhas_trajeto} de {df_acidentes_padronizado.shape[0]} linhas totais."
)

# removendo as linhas que possuem "trajeto" na coluna Classificação
df_acidentes_padronizado = df_acidentes_padronizado[
    ~df_acidentes_padronizado["classificacao"].str.contains("trajeto")
]

print(
    f"Quantidade de linhas após a remoção: {df_acidentes_padronizado.shape[0]}")

### Removendo acidentes que ocorreram em alguma empresa que não é a Eletrosul

In [ ]:
df_acidentes_padronizado = df_acidentes_padronizado[
    df_acidentes_padronizado["empresa"] == "cgt eletrosul"
]

print(
    f"Quantidade de linhas após a remoção: {df_acidentes_padronizado.shape[0]}")

## Removendo acidentes sem ID

In [ ]:
qtd_nans_id = df_acidentes_padronizado["id"].isna().sum()

print("Quantidade de NaNs na coluna ID:", qtd_nans_id)

df_acidentes_padronizado = df_acidentes_padronizado[
    (~df_acidentes_padronizado["id"].isna()) & (
        df_acidentes_padronizado["id"] != '-')
]

## Eliminação das linhas de acidentes irrelevantes e de predisposição física

In [ ]:
print(
    f"Quantidade de linhas antes a remoção: {df_acidentes_padronizado.shape[0]}")

# Valores da coluna "Indice acidente" dos acidentes que foram julgados irrelevantes
acidentes_irrelevantes = [56, 102, 171, 206,
                          213, 294, 489, 667, 703, 788, 810, 844]

# Valores da coluna "Indice acidente" dos acidentes com causa relacionada à predisposição física do empregado
acidentes_por_predisposicao_fisica = [
    86, 179, 182, 205, 250, 257, 341, 443, 496]

# Eliminando acidentes_irrelevantes
df_acidentes_padronizado = df_acidentes_padronizado[~df_acidentes_padronizado["indice acidente"].isin(
    acidentes_irrelevantes)]

# Eliminando acidentes_por_predisposicao_fisica
df_acidentes_padronizado = df_acidentes_padronizado[~df_acidentes_padronizado["indice acidente"].isin(
    acidentes_por_predisposicao_fisica)]

print(
    f"Quantidade de linhas após a remoção: {df_acidentes_padronizado.shape[0]}")

### Selecionando quais colunas ficarão

In [ ]:
columns = ['classificacao', 'id', 'data', 'descricao', 'latitude', 'longitude']
df_acidentes_padronizado = df_acidentes_padronizado[columns]

### Salvando o novo dataset

In [ ]:
# Salva o dataset no folder modificados
df_acidentes_padronizado.to_csv(
    pathUtil + "acidentes/" + "acidentes_preparado.csv",
    sep="@",
    encoding="utf-8",
    index=False,
)